# Make DC2 DDF catalog for sprinkler input

As part of the new sprinkler we want to just take in a catalog and create a general output truth catalog. To do the matching for this new sprinkler we still need to know what's in the simulated data that we will replace. But instead of using an instance catalog as input we need a general catalog that includes the galaxies and AGN with variability parameters for the AGN. The AGN information as well as SED information is cached in separate databases, but we can put them all together to make one file with all the necessary information for all objects in the DC2 DDF field.

In [1]:
import os
import pandas as pd
import numpy as np
import GCRCatalogs
import matplotlib as mpl
import matplotlib.pyplot as plt
from GCR import GCRQuery
from lsst.utils import getPackageDir
from sklearn.neighbors import NearestNeighbors

## Load cosmoDC2 galaxies in the DDF

In [2]:
catalog = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image_addon_knots')

In [3]:
quantity_list = ['galaxy_id',
                 'ra',
                 'dec',
                 'redshift_true',
                 'shear_1',
                 'shear_2_phosim',
                 'convergence',
                 'position_angle_true',
                 'size_disk_true',
                 'size_minor_disk_true',
                 'size_bulge_true',
                 'size_minor_bulge_true',
                 'sersic_disk',
                 'sersic_bulge',
                 'stellar_mass_disk',
                 'stellar_mass_bulge',
                 'stellar_mass',
                 'totalStarFormationRate',
                  'Mag_true_u_lsst_z0',
#                  'Mag_true_g_lsst_z0',
                  'Mag_true_r_lsst_z0',
#                  'Mag_true_i_lsst_z0',
                ]

In [4]:
data = catalog.get_quantities(quantity_list, filters=['ra < 53.755', 'ra > 52.495',
                                                      'dec < -27.55', 'dec > -28.65'])


KeyboardInterrupt



In [ ]:
data

### Get healpix ids

We will need these to open the correct SED files.

In [ ]:
import healpy
def get_healpix_id(ra, dec):
    pix_id = healpy.ang2pix(32, ra, dec, nest=False, lonlat=True)
    return pix_id

In [ ]:
len(data['ra'])

In [ ]:
healpix_ids = get_healpix_id(data['ra'], data['dec'])

In [ ]:
healpix_ids

### Put galaxy information into pandas dataframe

In [ ]:
galaxy_df = pd.DataFrame(data)

In [ ]:
galaxy_df.head()

In [ ]:
galaxy_df['healpix_id'] = healpix_ids

In [ ]:
galaxy_df.head()

### Match to a galaxy type

Fit to type from Goldstein et al. catalog. The Goldstein et al.
catalog labels host galaxies as one of
three types: 'elliptical', 'sc', or 'starburst'.
We will assign labels to potential host galaxies
in the DDF to better match the catalog.


#### Only keep galaxies above mass cut at 10^8 solar masses

Focus on larger galaxies to better match mass distribution in Mannucci et al. (2005) paper we will use to estimate SN rate.

In [ ]:
mass_cut = np.where(np.log10(galaxy_df['stellar_mass']) >= 8.)[0]

In [ ]:
plt.hist(np.log10(galaxy_df['stellar_mass'][mass_cut]))
plt.title('Stellar Mass >= 10^8 Solar Masses')
plt.xlabel('Stellar Mass (Log(Solar Masses))')
plt.ylabel('Galaxy Count')

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(np.log10(galaxy_df['stellar_mass'][::10]),
            np.log10(data['totalStarFormationRate'][::10]),
            alpha=0.05, c=data['Mag_true_u_lsst_z0'][::10] - data['Mag_true_r_lsst_z0'][::10])
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='true_u_lsst_z0 - true_r_lsst_z0', size=18)
cbar.set_alpha(1)
cbar.draw_all()
plt.xlabel('Stellar Mass', size=18)
plt.ylabel('SFR', size=18)
plt.xticks(size=18)
plt.yticks(size=18)

#### Divide galaxy types by regions of stellar mass vs star formation rate

Highest mass and lowest star formation rate will be ellipticals. Lowest mass and highest star formation rate will be starburst.

In [ ]:
x_pts = np.arange(8, 12, 0.005)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(np.log10(data['stellar_mass'][mass_cut]),
            np.log10(data['totalStarFormationRate'][mass_cut]),
            alpha=0.05, c=data['Mag_true_u_lsst_z0'][mass_cut] - data['Mag_true_r_lsst_z0'][mass_cut])
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='true_u_lsst_z0 - true_r_lsst_z0', size=18)
cbar.set_alpha(1)
cbar.draw_all()
plt.xlabel('Stellar Mass', size=18)
plt.ylabel('SFR', size=18)
plt.xticks(size=18)
plt.yticks(size=18)
plt.plot(x_pts, -12.3 + x_pts*2)
plt.plot(x_pts, -10 + x_pts*2)
plt.title('Proposed Galaxy Type Boundaries')

In [ ]:
# We keep all galaxies even those below the stellar mass cut and just assign those
# below the minimum mass to type = -99 so we can identify them later
gal_type = np.ones(len(data['stellar_mass']))*-99

In [ ]:
cut_val = np.log10(data['stellar_mass'][mass_cut])*2 - np.log10(data['totalStarFormationRate'][mass_cut])

In [ ]:
gal_type[mass_cut[np.where(cut_val <= 10)]] = 2. #Starburst
gal_type[mass_cut[np.where((cut_val > 10) & (cut_val < 12.3))]] = 1. #SC
gal_type[mass_cut[np.where(cut_val >= 12.3)]] = 0. #Elliptical

In [ ]:
gal_type[:10]

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(np.log10(data['stellar_mass'][mass_cut]),
            np.log10(data['totalStarFormationRate'][mass_cut]),
            alpha=0.05, c=gal_type[mass_cut])
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
cbar.set_ticks([0, 1, 2])
cbar.set_ticklabels(['Elliptical', 'Sc', 'Starburst'])
cbar.set_label(label='Galaxy Type Assigned', size=18)
cbar.set_alpha(1)
cbar.draw_all()
plt.xlabel('Stellar Mass', size=18)
plt.ylabel('SFR', size=18)
plt.xticks(size=18)
plt.yticks(size=18)
plt.plot(x_pts, -12.3 + x_pts*2)
plt.plot(x_pts, -10 + x_pts*2)

In [ ]:
n, bins, _ = plt.hist(np.log10(data['stellar_mass'])[np.where(gal_type==0)], label='Ellip', histtype='step', range=(8.0, 12.), bins=12, density=True)
plt.hist(np.log10(data['stellar_mass'])[np.where(gal_type==1)], label='Sc', histtype='step', bins=bins, density=True)
plt.hist(np.log10(data['stellar_mass'])[np.where(gal_type==2)], label='Starburst', histtype='step', bins=bins, density=True)
plt.xlabel('Stellar Mass (Log(Solar Masses))')
plt.ylabel('Galaxy Count')
plt.legend()
plt.title('Galaxy Type Assignments')

In [ ]:
kinney_types = ['kinney-elliptical', 'kinney-sc', 'kinney-starburst']

In [ ]:
kinney_type_dict = {-99:'None', 0:'kinney-elliptical', 1:'kinney-sc', 2:'kinney-starburst'}

In [ ]:
kinney_types_dc2 = []
for type_id in gal_type:
    kinney_types_dc2.append(kinney_type_dict[int(type_id)])

In [ ]:
np.unique(kinney_types_dc2, return_counts=True)

In [ ]:
galaxy_df['host_type'] = kinney_types_dc2

In [ ]:
galaxy_df.head(10)

## Get AGN database and save to pandas

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('/global/projecta/projectdirs/lsst/groups/SSim/DC2/cosmoDC2_v1.1.4/agn_db_mbh7_mi30_sf4.db')

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [ ]:
print(cur.fetchall())

In [ ]:
agn_df = pd.read_sql_query("SELECT * from agn_params", conn)

In [ ]:
agn_df.head()

In [ ]:
len(agn_df)

### Join databases on `galaxy_id`

In [39]:
galaxy_agn_join = galaxy_df.join(agn_df.set_index('galaxy_id'), on='galaxy_id')

In [40]:
galaxy_agn_join = galaxy_agn_join.rename(columns={'magNorm':'magNorm_agn', 'varParamStr':'varParamStr_agn'})

In [41]:
galaxy_agn_join.head(10)

,shear_2_phosim,redshift_true,galaxy_id,size_minor_disk_true,stellar_mass_disk,stellar_mass_bulge,size_bulge_true,shear_1,Mag_true_r_lsst_z0,ra,...,position_angle_true,size_disk_true,totalStarFormationRate,sersic_disk,Mag_true_u_lsst_z0,healpix_id,host_type,htmid_8,magNorm_agn,varParamStr_agn
0,-0.000000,0.020617,1875000009,0.733156,5.135591e+06,7.396574e+06,0.692787,-0.000000,-15.099893,52.545849,...,3.872094,1.390374,2.623878e+06,1.0,-13.647583,8914,None,NaN,NaN,NaN
1,-0.000000,0.019394,1875000034,0.670891,6.849424e+06,1.097576e+06,0.739462,-0.000000,-14.982487,52.623324,...,151.776670,1.494187,4.610795e+06,1.0,-13.789320,8914,None,NaN,NaN,NaN
2,-0.000000,0.017482,1875000035,0.970572,1.003473e+06,0.000000e+00,0.913097,-0.000000,-13.019890,52.681314,...,50.902277,2.377272,1.059278e+04,1.0,-11.321993,8914,None,NaN,NaN,NaN
3,-0.000000,0.017347,1875000055,3.347761,3.920008e+07,9.869368e+06,1.537620,-0.000000,-16.026091,52.581461,...,174.550510,3.411983,1.586908e+07,1.0,-14.870754,8914,None,NaN,NaN,NaN
4,-0.000023,0.046867,1875000072,1.000541,7.509686e+06,1.162219e+06,0.602592,-0.000026,-14.948131,52.737179,...,117.095584,1.470491,5.564104e+06,1.0,-13.690415,8914,None,NaN,NaN,NaN
5,0.000001,0.047066,1875000080,27.935360,2.394919e+10,3.178831e+11,18.749584,0.000002,-23.226475,52.604145,...,94.870050,29.920483,4.033726e+05,1.0,-20.898287,8914,kinney-elliptical,147051.0,17.633345,"{""m"": ""applyAgn"", ""p"": {""seed"": 4614010, ""agn_..."
6,0.000018,0.047066,1875000081,2.017076,5.922428e+08,4.518640e+09,1.244547,-0.000010,-19.433334,52.560131,...,80.927515,2.284488,3.479801e+04,1.0,-17.151030,8914,kinney-elliptical,NaN,NaN,NaN
7,0.000013,0.047066,1875000085,1.216847,1.012820e+08,1.160316e+08,1.079106,-0.000010,-17.189556,52.940460,...,169.095427,2.133967,2.452688e+08,1.0,-15.417778,8914,kinney-starburst,NaN,NaN,NaN
8,0.000034,0.047066,1875000087,2.871373,1.220466e+10,4.468104e+10,2.911081,-0.000037,-21.450287,52.648149,...,16.066011,4.743254,2.707374e+07,1.0,-19.152988,8914,kinney-elliptical,147051.0,22.150698,"{""m"": ""applyAgn"", ""p"": {""seed"": 2957413, ""agn_..."
9,0.000026,0.047066,1875000088,1.126129,3.865890e+07,1.210977e+07,0.544203,-0.000037,-16.182764,52.652109,...,48.715094,1.142435,5.657011e+03,1.0,-14.127932,8914,None,NaN,NaN,NaN


### Calculate i-band mags for AGN

In [42]:
from lsst.sims.photUtils import Sed, BandpassDict, getImsimFluxNorm, Bandpass
from lsst.utils import getPackageDir
from copy import deepcopy

In [43]:
lsst_bp_dict = BandpassDict.loadTotalBandpassesFromFiles()

In [44]:
os.listdir(os.path.join(getPackageDir('SIMS_SED_LIBRARY'), 'agnSED'))

['agn.spec.gz']

In [45]:
agn_sed = Sed()
agn_sed.readSED_flambda(os.path.join(getPackageDir('SIMS_SED_LIBRARY'), 'agnSED', 'agn.spec.gz'))

In [46]:
mag_i_agn = []
for agn_data in galaxy_agn_join[['magNorm_agn', 'redshift_true']].values:
    agn_mag, agn_redshift = agn_data
    if np.isnan(agn_mag):
        mag_i_agn.append(np.nan)
        continue
    agn_copy = deepcopy(agn_sed)
    flux_norm = getImsimFluxNorm(agn_copy, agn_mag)
    agn_copy.multiplyFluxNorm(flux_norm)
    agn_copy.redshiftSED(agn_redshift, dimming=True)
    mag_i_agn.append(agn_copy.calcMag(lsst_bp_dict['i']))

In [47]:
galaxy_agn_join['mag_i_agn'] = mag_i_agn

In [48]:
galaxy_agn_join.head(10)

,shear_2_phosim,redshift_true,galaxy_id,size_minor_disk_true,stellar_mass_disk,stellar_mass_bulge,size_bulge_true,shear_1,Mag_true_r_lsst_z0,ra,...,size_disk_true,totalStarFormationRate,sersic_disk,Mag_true_u_lsst_z0,healpix_id,host_type,htmid_8,magNorm_agn,varParamStr_agn,mag_i_agn
0,-0.000000,0.020617,1875000009,0.733156,5.135591e+06,7.396574e+06,0.692787,-0.000000,-15.099893,52.545849,...,1.390374,2.623878e+06,1.0,-13.647583,8914,None,NaN,NaN,NaN,NaN
1,-0.000000,0.019394,1875000034,0.670891,6.849424e+06,1.097576e+06,0.739462,-0.000000,-14.982487,52.623324,...,1.494187,4.610795e+06,1.0,-13.789320,8914,None,NaN,NaN,NaN,NaN
2,-0.000000,0.017482,1875000035,0.970572,1.003473e+06,0.000000e+00,0.913097,-0.000000,-13.019890,52.681314,...,2.377272,1.059278e+04,1.0,-11.321993,8914,None,NaN,NaN,NaN,NaN
3,-0.000000,0.017347,1875000055,3.347761,3.920008e+07,9.869368e+06,1.537620,-0.000000,-16.026091,52.581461,...,3.411983,1.586908e+07,1.0,-14.870754,8914,None,NaN,NaN,NaN,NaN
4,-0.000023,0.046867,1875000072,1.000541,7.509686e+06,1.162219e+06,0.602592,-0.000026,-14.948131,52.737179,...,1.470491,5.564104e+06,1.0,-13.690415,8914,None,NaN,NaN,NaN,NaN
5,0.000001,0.047066,1875000080,27.935360,2.394919e+10,3.178831e+11,18.749584,0.000002,-23.226475,52.604145,...,29.920483,4.033726e+05,1.0,-20.898287,8914,kinney-elliptical,147051.0,17.633345,"{""m"": ""applyAgn"", ""p"": {""seed"": 4614010, ""agn_...",17.251804
6,0.000018,0.047066,1875000081,2.017076,5.922428e+08,4.518640e+09,1.244547,-0.000010,-19.433334,52.560131,...,2.284488,3.479801e+04,1.0,-17.151030,8914,kinney-elliptical,NaN,NaN,NaN,NaN
7,0.000013,0.047066,1875000085,1.216847,1.012820e+08,1.160316e+08,1.079106,-0.000010,-17.189556,52.940460,...,2.133967,2.452688e+08,1.0,-15.417778,8914,kinney-starburst,NaN,NaN,NaN,NaN
8,0.000034,0.047066,1875000087,2.871373,1.220466e+10,4.468104e+10,2.911081,-0.000037,-21.450287,52.648149,...,4.743254,2.707374e+07,1.0,-19.152988,8914,kinney-elliptical,147051.0,22.150698,"{""m"": ""applyAgn"", ""p"": {""seed"": 2957413, ""agn_...",21.769157
9,0.000026,0.047066,1875000088,1.126129,3.865890e+07,1.210977e+07,0.544203,-0.000037,-16.182764,52.652109,...,1.142435,5.657011e+03,1.0,-14.127932,8914,None,NaN,NaN,NaN,NaN


## Load cached SED information

In [49]:
print(np.unique(healpix_ids))

[8914 8915 9043]


In [50]:
sed_dir = '/global/projecta/projectdirs/lsst/groups/SSim/DC2/cosmoDC2_v1.1.4/sedLookup'

In [51]:
import h5py

In [52]:
f = h5py.File(os.path.join(sed_dir, 'sed_fit_8914.h5'))

In [53]:
! ls /global/projecta/projectdirs/lsst/groups/SSim/DC2/cosmoDC2_v1.1.4/sedLookup

sed_fit_10066.h5  sed_fit_10449.h5  sed_fit_9171.h5  sed_fit_9560.h5
sed_fit_10067.h5  sed_fit_10450.h5  sed_fit_9172.h5  sed_fit_9561.h5
sed_fit_10068.h5  sed_fit_10451.h5  sed_fit_9173.h5  sed_fit_9562.h5
sed_fit_10069.h5  sed_fit_10452.h5  sed_fit_9174.h5  sed_fit_9681.h5
sed_fit_10070.h5  sed_fit_8786.h5   sed_fit_9175.h5  sed_fit_9682.h5
sed_fit_10071.h5  sed_fit_8787.h5   sed_fit_9176.h5  sed_fit_9683.h5
sed_fit_10072.h5  sed_fit_8788.h5   sed_fit_9177.h5  sed_fit_9684.h5
sed_fit_10073.h5  sed_fit_8789.h5   sed_fit_9178.h5  sed_fit_9685.h5
sed_fit_10074.h5  sed_fit_8790.h5   sed_fit_9298.h5  sed_fit_9686.h5
sed_fit_10193.h5  sed_fit_8791.h5   sed_fit_9299.h5  sed_fit_9687.h5
sed_fit_10194.h5  sed_fit_8792.h5   sed_fit_9300.h5  sed_fit_9688.h5
sed_fit_10195.h5  sed_fit_8793.h5   sed_fit_9301.h5  sed_fit_9689.h5
sed_fit_10196.h5  sed_fit_8794.h5   sed_fit_9302.h5  sed_fit_9690.h5
sed_fit_10197.h5  sed_fit_8913.h5   sed_fit_9303.h5  sed_fit_9810.h5
sed_fit_10198.h5  sed_fit_8914.h5 

In [54]:
list(f.keys())

['bulge_av',
 'bulge_fluxes',
 'bulge_magnorm',
 'bulge_rv',
 'bulge_sed',
 'dec',
 'disk_av',
 'disk_fluxes',
 'disk_magnorm',
 'disk_rv',
 'disk_sed',
 'galaxy_id',
 'ra',
 'redshift',
 'sed_names',
 'tot_fluxes']

In [55]:
sed_df = pd.DataFrame([], columns=['bulge_av',
 'bulge_rv',
 'bulge_sed',
 'disk_av',
 'disk_rv',
 'disk_sed',
 'galaxy_id',])

In [58]:
for hpix in [8914, 8915, 9043]:
    f = h5py.File(os.path.join(sed_dir, 'sed_fit_%i.h5' % hpix))
    
    sed_hpix_df = pd.DataFrame([])

    band_list = ['u', 'g', 'r', 'i', 'z', 'y']
    sed_names = f['sed_names'][()]
    for key in list(f.keys()):
        print(key, f[key].len())
        if (key.endswith('fluxes') or key.endswith('magnorm')):
            key_data = f[key][()]
            for i in range(6):
                sed_hpix_df[str(key + '_' + band_list[i])] = key_data[i]
        elif key in ['sed_names', 'ra', 'dec']:
            continue
        elif key.endswith('sed'):
            sed_hpix_df[key] = [sed_names[idx] for idx in f[key][()]]
        else:
            sed_hpix_df[key] = f[key][()]
            
    sed_df = pd.concat([sed_df, sed_hpix_df])

bulge_av 17251417
bulge_fluxes 6
bulge_magnorm 6
bulge_rv 17251417
bulge_sed 17251417
dec 17251417
disk_av 17251417
disk_fluxes 6
disk_magnorm 6
disk_rv 17251417
disk_sed 17251417
galaxy_id 17251417
ra 17251417
redshift 17251417
sed_names 959
tot_fluxes 6


/global/common/cori_cle7/software/jupyter/19-09/lib/python3.7/site-packages/ipykernel/__main__.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



bulge_av 17215541
bulge_fluxes 6
bulge_magnorm 6
bulge_rv 17215541
bulge_sed 17215541
dec 17215541
disk_av 17215541
disk_fluxes 6
disk_magnorm 6
disk_rv 17215541
disk_sed 17215541
galaxy_id 17215541
ra 17215541
redshift 17215541
sed_names 959
tot_fluxes 6
bulge_av 17182059
bulge_fluxes 6
bulge_magnorm 6
bulge_rv 17182059
bulge_sed 17182059
dec 17182059
disk_av 17182059
disk_fluxes 6
disk_magnorm 6
disk_rv 17182059
disk_sed 17182059
galaxy_id 17182059
ra 17182059
redshift 17182059
sed_names 959
tot_fluxes 6


In [59]:
sed_df = sed_df.reset_index(drop=True)

In [60]:
sed_df.head()

,bulge_av,bulge_fluxes_g,bulge_fluxes_i,bulge_fluxes_r,bulge_fluxes_u,bulge_fluxes_y,bulge_fluxes_z,bulge_magnorm_g,bulge_magnorm_i,bulge_magnorm_r,...,disk_rv,disk_sed,galaxy_id,redshift,tot_fluxes_g,tot_fluxes_i,tot_fluxes_r,tot_fluxes_u,tot_fluxes_y,tot_fluxes_z
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,inf,inf,inf,...,2.0,b'galaxySED/Burst.20E09.002Z.spec.gz',1875000000,0.015724,0.000889,0.001194,0.001087,0.000368,0.001267,0.001224
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,inf,inf,inf,...,3.2,b'galaxySED/Burst.12E09.02Z.spec.gz',1875000001,0.022920,0.000017,0.000025,0.000022,0.000007,0.000027,0.000026
2,0.3,0.000737,0.002257,0.001561,0.000156,0.003474,0.002914,16.218253,16.142116,16.149535,...,4.0,b'galaxySED/Const.12E10.04Z.spec.gz',1875000002,0.022485,0.002766,0.006330,0.004731,0.000975,0.009214,0.007852
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,inf,inf,inf,...,2.0,b'galaxySED/Inst.12E10.002Z.spec.gz',1875000003,0.022348,0.000119,0.000222,0.000183,0.000039,0.000252,0.000239
4,0.1,0.000350,0.000700,0.000563,0.000111,0.000813,0.000764,17.301761,17.333419,17.320859,...,2.0,b'galaxySED/Exp.10E09.02Z.spec.gz',1875000004,0.022606,0.000351,0.000700,0.000563,0.000112,0.000814,0.000765


In [61]:
len(sed_df)

51649017

## Join databases on `galaxy_id`

In [62]:
full_join = galaxy_agn_join.join(sed_df.set_index('galaxy_id'), on='galaxy_id')

In [63]:
full_join.head(10)

,shear_2_phosim,redshift_true,galaxy_id,size_minor_disk_true,stellar_mass_disk,stellar_mass_bulge,size_bulge_true,shear_1,Mag_true_r_lsst_z0,ra,...,disk_magnorm_z,disk_rv,disk_sed,redshift,tot_fluxes_g,tot_fluxes_i,tot_fluxes_r,tot_fluxes_u,tot_fluxes_y,tot_fluxes_z
0,-0.000000,0.020617,1875000009,0.733156,5.135591e+06,7.396574e+06,0.692787,-0.000000,-15.099893,52.545849,...,20.477642,4.0,b'galaxySED/Exp.25E09.02Z.spec.gz',0.020709,0.000032,0.000061,0.000049,0.000012,0.000073,0.000068
1,-0.000000,0.019394,1875000034,0.670891,6.849424e+06,1.097576e+06,0.739462,-0.000000,-14.982487,52.623324,...,19.941111,3.8,b'galaxySED/Exp.25E09.02Z.spec.gz',0.019261,0.000037,0.000060,0.000051,0.000016,0.000067,0.000064
2,-0.000000,0.017482,1875000035,0.970572,1.003473e+06,0.000000e+00,0.913097,-0.000000,-13.019890,52.681314,...,21.916558,2.0,b'galaxySED/Exp.80E09.02Z.spec.gz',0.017699,0.000006,0.000013,0.000010,0.000002,0.000016,0.000015
3,-0.000000,0.017347,1875000055,3.347761,3.920008e+07,9.869368e+06,1.537620,-0.000000,-16.026091,52.581461,...,18.633536,2.9,b'galaxySED/Const.10E10.02Z.spec.gz',0.017156,0.000121,0.000197,0.000166,0.000055,0.000226,0.000213
4,-0.000023,0.046867,1875000072,1.000541,7.509686e+06,1.162219e+06,0.602592,-0.000026,-14.948131,52.737179,...,21.812307,4.0,b'galaxySED/Exp.25E09.02Z.spec.gz',0.044630,0.000005,0.000010,0.000008,0.000002,0.000011,0.000011
5,0.000001,0.047066,1875000080,27.935360,2.394919e+10,3.178831e+11,18.749584,0.000002,-23.226475,52.604145,...,16.875877,3.8,b'galaxySED/Burst.10E10.1Z.spec.gz',0.044378,0.007280,0.021646,0.015266,0.001589,0.032623,0.027467
6,0.000018,0.047066,1875000081,2.017076,5.922428e+08,4.518640e+09,1.244547,-0.000010,-19.433334,52.560131,...,20.195039,4.0,b'galaxySED/Inst.10E10.1Z.spec.gz',0.043418,0.000224,0.000662,0.000464,0.000051,0.000970,0.000824
7,0.000013,0.047066,1875000085,1.216847,1.012820e+08,1.160316e+08,1.079106,-0.000010,-17.189556,52.940460,...,20.075499,4.0,b'galaxySED/Const.12E10.04Z.spec.gz',0.045444,0.000032,0.000085,0.000059,0.000010,0.000135,0.000114
8,0.000034,0.047066,1875000087,2.871373,1.220466e+10,4.468104e+10,2.911081,-0.000037,-21.450287,52.648149,...,16.962426,4.0,b'galaxySED/Exp.62E09.1Z.spec.gz',0.044377,0.001442,0.004172,0.002977,0.000324,0.006293,0.005263
9,0.000026,0.047066,1875000088,1.126129,3.865890e+07,1.210977e+07,0.544203,-0.000037,-16.182764,52.652109,...,21.051351,4.0,b'galaxySED/Exp.80E09.02Z.spec.gz',0.045954,0.000012,0.000032,0.000023,0.000003,0.000044,0.000039


## Save file 

In [2]:
import pickle

In [66]:
with open('../data/full_ddf.pkl', 'wb') as f:
    pickle.dump(full_join, f)

### Add Milky Way Av, Rv values

In [4]:
with open('../data/full_ddf.pkl', 'rb') as f:
    full_join = pickle.load(f)

In [19]:
from lsst.sims.catUtils.dust.EBV import EBVbase
import numpy as np

In [20]:
ebvObj = EBVbase()

In [28]:
ebv_vals = ebvObj.calculateEbv(equatorialCoordinates=np.array([full_join['ra'], full_join['dec']]))

In [30]:
av_vals = 3.1 * ebv_vals

In [33]:
full_join['av_mw'] = av_vals
full_join['rv_mw'] = 3.1

In [34]:
with open('../data/full_ddf.pkl', 'wb') as f:
    pickle.dump(full_join, f)

In [35]:
full_join.columns

Index(['shear_2_phosim', 'redshift_true', 'galaxy_id', 'size_minor_disk_true',
       'stellar_mass_disk', 'stellar_mass_bulge', 'size_bulge_true', 'shear_1',
       'Mag_true_r_lsst_z0', 'ra', 'sersic_bulge', 'size_minor_bulge_true',
       'dec', 'convergence', 'stellar_mass', 'position_angle_true',
       'size_disk_true', 'totalStarFormationRate', 'sersic_disk',
       'Mag_true_u_lsst_z0', 'healpix_id', 'host_type', 'htmid_8',
       'magNorm_agn', 'varParamStr_agn', 'mag_i_agn', 'bulge_av',
       'bulge_fluxes_g', 'bulge_fluxes_i', 'bulge_fluxes_r', 'bulge_fluxes_u',
       'bulge_fluxes_y', 'bulge_fluxes_z', 'bulge_magnorm_g',
       'bulge_magnorm_i', 'bulge_magnorm_r', 'bulge_magnorm_u',
       'bulge_magnorm_y', 'bulge_magnorm_z', 'bulge_rv', 'bulge_sed',
       'disk_av', 'disk_fluxes_g', 'disk_fluxes_i', 'disk_fluxes_r',
       'disk_fluxes_u', 'disk_fluxes_y', 'disk_fluxes_z', 'disk_magnorm_g',
       'disk_magnorm_i', 'disk_magnorm_r', 'disk_magnorm_u', 'disk_magnorm_y',